In [1]:
import sys
sys.executable

'D:\\exec-in-cmd\\miniconda3\\envs\\min_ds-env\\python.exe'

In [2]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian
import os
import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không
from selenium import webdriver

In [3]:
#PATH = 'C:\Program Files (x86)\chromedriver.exe'

def GetUser():
    
    UsersLink = []
    UsersName = []
    Users = []
    
    url = 'https://soundcloud.com/discover/sets/charts-top:all-music:vn'
    #url = 'https://soundcloud.com/discover/sets/charts-top:hiphoprap:vn'
    
    browser = webdriver.Chrome(executable_path="chromedriver.exe")
    browser.get(url)
    time.sleep(3)
    
    # To make it load more scroll to the bottom of the page (repeat if you want to)
    
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match = False
    while(match == False):
        lastCount = lenOfPage
        time.sleep(3)
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount == lenOfPage:
                match = True
                    
    html_source = browser.page_source
    browser.quit()
    soup = BeautifulSoup(html_source, 'html.parser')
    for id in soup.findAll('a', {'class' : 'trackItem__username sc-link-light sc-link-secondary'}):
        UsersLink.append(id.get('href'))
        UsersName.append(id.text)
        
    Users.append(url)
    Users.append(UsersLink)
    Users.append(UsersName)
    return Users

def GetTracks(Users):
    TracksOfAllUsers = []
    UserName = []
    BoxTracks = []
    
    for i in range(len(Users[1])):
        TracksOfOneUser = []
        UserName.append(Users[2][i])#lay ten user
        
        url = 'https://soundcloud.com'
        url = url + str(Users[1][i]) + '/tracks'
    
        browser = webdriver.Chrome(executable_path="chromedriver.exe")
        browser.get(url)
        time.sleep(3)

        # To make it load more scroll to the bottom of the page (repeat if you want to)
        
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match = False
        while(match == False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage:
                match = True
                
        html_source = browser.page_source
        browser.quit()
        
        soup = BeautifulSoup(html_source, 'html.parser')
        check = soup.findAll('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
        if check == []:
            TracksOfOneUser.append('None')
        else:
            for id in check:
                TracksOfOneUser.append(id.text)#lay track cua 1 user
                
        BoxTracks.append(TracksOfOneUser)
    
    TracksOfAllUsers.append(UserName)
    TracksOfAllUsers.append(BoxTracks)
    return TracksOfAllUsers

def GetPlaylist(Users):
    PlaylistOfAllUsers = []
    UserName = []
    PlaylistName = []
    BoxTracks = []
    
    for i in range(len(Users[1])):
        UserName.append(Users[2][i])#lay ten username
        
        url = 'https://soundcloud.com'
        url = url + str(Users[1][i]) + '/sets' #lay duong dan den playlist
        
        browser = webdriver.Chrome(executable_path="chromedriver.exe")
        browser.get(url)
        time.sleep(3)

        # To make it load more scroll to the bottom of the page (repeat if you want to)
        
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        match = False
        while(match == False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage:
                match = True
                
        html_source = browser.page_source
        browser.quit()
        
        soup = BeautifulSoup(html_source, 'html.parser')
        check = soup.findAll('div', {'class' : 'sound__content'})
        if check == []:
            PlaylistName.append('None')
            BoxTracks.append('None')
        else:        
            for id in check:
                TracksOfOnePlaylist = []
                
                Title = id.find('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
                PlaylistName.append(Title.text)

                Tracks = id.findAll('span', {'class' : 'compactTrackListItem__trackTitle sc-text-primary sc-text-h4'})
                for Track in Tracks:
                    TracksOfOnePlaylist.append(Track.text)
                BoxTracks.append(TracksOfOnePlaylist)
                
    PlaylistOfAllUsers.append(UserName)
    PlaylistOfAllUsers.append(PlaylistName)
    PlaylistOfAllUsers.append(BoxTracks)
    return PlaylistOfAllUsers

def writeUsersToCsv(List2D, filename):
    df = pd.DataFrame(List2D, columns = ['Url', 'Users Link', 'User Name'])
    df.to_csv(filename, index = True, sep = '\t')
    
def writeTracksToCsv(List2D, filename):
    df = pd.DataFrame(List2D, columns = ['User Name', 'Tracks Of One User'])
    df.to_csv(filename, index = True, sep = '\t')

def writePlaylistsToCsv(List2D, filename):
    df = pd.DataFrame(List2D, columns = ['User Name', 'Playlists Of One User', 'Tracks Of One Playlist'])
    df.to_csv(filename, index = True, sep = '\t')

In [4]:
Users = GetUser()
#Tracks = GetTracks(Users)
#Playlists = GetPlaylist(Users)
#print(Tracks)
#writeUsersToCsv(Users, './Crawl_data/users.csv')
#writeTracksToCsv(Tracks, './Crawl_data/tracks.csv')
#writePlaylistsToCsv(Playlists, './Crawl_data/playlists.csv')
#print(len(Users[1]))
print(len(Users[2]))
#GetTracks(Users)
#print(len(GetPlaylist(Users)))

48


In [5]:
def TestGetTracks(Users):
    url = 'https://soundcloud.com/winnodamidside'
    browser = webdriver.Chrome(executable_path="chromedriver.exe")
    browser.get(url)
    time.sleep(3)

    # To make it load more scroll to the bottom of the page (repeat if you want to)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    html_source = browser.page_source
    browser.quit()
        
    soup = BeautifulSoup(html_source, 'html.parser')
    for id in soup.findAll('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'}):
        if id == []:
            print('None')
        else:
            print(id.text)

def TestGetPlaylist(Users):
    url = 'https://soundcloud.com/nguy-n-to-n-401524044/sets'
    
    browser = webdriver.Chrome(executable_path="chromedriver.exe")
    browser.get(url)
    time.sleep(3)

    # To make it load more scroll to the bottom of the page (repeat if you want to)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    html_source = browser.page_source
    browser.quit()
        
    soup = BeautifulSoup(html_source, 'html.parser')
    for id in soup.findAll('div', {'class' : 'sound__content'}):
        Title = id.find('a', {'class' : 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})
        print(Title.text)
        Tracks = id.findAll('span', {'class' : 'compactTrackListItem__trackTitle sc-text-primary sc-text-h4'})
        for Track in Tracks:
            print(Track.text)
        print('\n')

In [7]:
TestGetTracks(Users)
TestGetPlaylist(Users)


Tell Ur Mom II - Winno (ft Heily)


Hai Tám Bốn - Winno


mua` xuan^ ma^t' em anh di bo^c' da^u` - Green x Ccmk


Bởi vì ai rồi cũng phải lớn khôn (CUZ)  _H$ Mẫn (prod.@soulju)


"Dongchenbat".flp


Lil Thang - pu ekvw Ft Hashone


Srry to my ladies + Young Bo5 prod. Icevxz


Dear Teacher - Robe


Body High - SMVL X Winno


Dạo bước cùng rồng vàng - Winno (SunXiX flip)


Fin - track 2: Khonghoitiec (No regret) [21DAMIXTAPE] (prod by. Origami)


Fancy - Winno (slowed + reverb)


Timemkoke1linenao * winno rmx *


Bốp 2 - Winno (Godthic dissin)


'tim em ko kó một đường nào' - Winno, Heily (slowed + reverb)


YungP - RobLucci (prod. $tarr)


PICK UP UR PHONE - GG$ KASIANEH


RỐI LOẠN NHÂN CÁCH ÁI KỶ! - POPBEN (REPLY S-FURY)


K-TLE & CHLLDISH - Redd Ocean


Remix slowed songs


    𝚈𝚎𝚞 5_𝚜𝚕𝚘𝚠𝚎𝚍+𝚛𝚎𝚟𝚎𝚛𝚋
  

    𝚃𝚞𝚘𝚗𝚐 𝚀𝚞𝚊𝚗_𝚜𝚕𝚘𝚠𝚎𝚍+𝚛𝚎𝚟𝚎𝚛𝚋
  

    𝚃𝚞𝚢 𝙰𝚖_𝚜𝚕𝚘𝚠𝚎𝚍+𝚛𝚎𝚟𝚎𝚛𝚋
  



Sad slowed songs


    𝚃𝚑𝚊𝚗𝚐 𝚍𝚒𝚎𝚗_𝙹𝚞𝚜𝚝𝚊𝚃𝚎𝚎 (𝚜𝚕𝚘𝚠𝚎𝚍+𝚛𝚎𝚟𝚎𝚛𝚋)
  

    𝙲𝚘 𝚗𝚑𝚞 𝚔𝚑𝚘𝚗𝚐 𝚌𝚘_𝙷𝚒𝚎𝚗 𝙷𝚘 (𝚜𝚕𝚘𝚠𝚎𝚍+𝚛𝚎𝚟𝚎𝚛𝚋)
  

    𝚌𝚑𝚊𝚖 𝚔